# Keras入門


## Kerasとは

Keras は元々はTheano、TensorFLowなど複数のフレームワークを扱いやすくする ラッパー として登場したライブラリでしたが、後にTensorFLowの 高レベルAPI として使われるようになりました。




TensorFLowに含まれる形のKerasであるtf.kerasを主に使っていきます。


**《ラッパーとは》**


ラッパーはもともとのプログラムの機能を利用して、より使いやすいものを提供します。TensorFlowはニューラルネットワークに必要な計算を効率的に行う機能を提供しますが、

初期のころはモデルを構築して学習を行うとなると手間がかかる部分もありました。そのため、TensorFlowをラップして、扱いやすくするKerasが登場しました。


**《高レベルAPIとは》**


大きな単位で機能を簡単に扱えるように作られたものが高レベルAPIです。対義語として、細かい単位で機能をいじれるが、扱いがその分大変な低レベルAPIがあります。


TensorFlow自体でもニューラルネットワークのモデル構築や学習を行いやすくするために、高レベルAPIの充実が進められています。tf.Kerasはそのひとつです。

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

### 複数の記述方法

Kerasでは簡素にニューラルネットワークが記述できます。

その書き方にはSequentialモデルとFunctional APIの2種類があります。それぞれを見ていきます。

In [3]:
# ANDゲート用のデータ
import numpy as np
# ANDゲートの学習データを用意
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

層のインスタンスをSequentialクラスのコンストラクタにリストで渡すことでモデルを定義します。

層のクラスについては以下のページにまとまっています。(DIVERのKeras入門参照)

ロジスティック回帰を作るために、全結合層のクラス、tf.keras.layers.Denseを使います。

引数に出力のユニット数、活性化関数、入力のユニット数を入れます。



In [4]:
model = tf.keras.Sequential([tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, input_shape=(2,))])
# 作成したモデルの確認
model.summary()

構造が記述できたら、モデルをコンパイルします。コンパイル時に損失関数と最適化手法、評価関数を指定します。損失関数は名前をstringで指定します。

ここでは2値分類のため、binary_crossentropyとなります。

------------------------------------

## 2値分類の場合、binary_crossentropy

## 多値分類の場合はcategorical_crossentropy、

## 回帰の場合はmean_squared_errorのようになります。

In [5]:
# 二値分類なのでlossは'binary_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

そして学習を行います。scikit-learn同様にfitメソッドを使う設計になっています。

verboseは学習過程の可視化方法のパラメータで、デフォルトの1ではバッチごとに更新されるプログレスバーが表示されます。

verboseが0の場合は表示を行わず、2の場合はエポック毎の表示になります。



In [6]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

===] - 0s 4ms/step - loss: 0.1154 - acc: 1.0000
Epoch 791/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1154 - acc: 1.0000
Epoch 792/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1151 - acc: 1.0000
Epoch 793/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1148 - acc: 1.0000
Epoch 794/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.1148 - acc: 1.0000
Epoch 795/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1146 - acc: 1.0000
Epoch 796/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.1144 - acc: 1.0000
Epoch 797/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1144 - acc: 1.0000
Epoch 798/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1142 - acc: 1.0000
Epoch 799/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1139 - acc: 1.0000
Epoch 800/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.1137 - acc: 1.0000
Ep

今は用意していませんが、検証用データがある場合は、引数validation_dataに与えることで、エポック毎の検証も可能です。



In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1,
                    validation_data=(x_train, y_train))#Validation_dataに検証データを与える

推定もscikit-learn同様にpredictメソッドを使います。

In [8]:
y_pred_proba = model.predict(x_train)[:, 0]
# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)
print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)

y_pred_proba [0.00167721 0.09540418 0.09603558 0.86960906]
y_pred [0 0 0 1]


結果がいらず、評価のみ行う場合はevaluateメソッドも便利です。

In [9]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train loss: 0.08565562218427658
Train accuracy: 1.0


## Sequentialモデルのもうひとつの書き方

Sequentialモデルでは、コンストラクタで層のクラスを渡さず、addメソッドを使って記述する方法もよく使われます。

In [10]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, input_shape=(2,)))

## 複数層の場合
ロジスティック回帰ではなく、2層のニューラルネットワークの場合は以下のように記述できます。

2層目以降はinput_shapeを与える必要がありません。tf.kerasが自動的に計算するためです。

In [11]:
model = tf.keras.Sequential([
            tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape=(2,)),
            tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)])

addメソッドを使えば次のようになります。

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape=(2,)))
model.add(tf.keras.layers.Dense(1, activation = tf.nn.sigmoid))

## Functional API

Functional APIを使えばより自由度の高いモデル構築が行えます。

Sequentialクラスの代わりにModelクラスを使用します。

入力から出力までの流れを記述していき、最後にModelクラスに入力層と出力層のインスタンスを渡します。

In [13]:
input_data = tf.keras.layers.Input(shape=(2,)) # 入力層
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(input_data) # 出力層

model = tf.keras.Model(inputs=input_data, outputs=output)

In [14]:
# 
# モデル構造の記述以降はSequentialモデルと全く同じです。
# 

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

====] - 0s 2ms/step - loss: 0.1438 - acc: 1.0000
Epoch 791/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1436 - acc: 1.0000
Epoch 792/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1435 - acc: 1.0000
Epoch 793/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1432 - acc: 1.0000
Epoch 794/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1429 - acc: 1.0000
Epoch 795/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1428 - acc: 1.0000
Epoch 796/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1424 - acc: 1.0000
Epoch 797/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1424 - acc: 1.0000
Epoch 798/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1421 - acc: 1.0000
Epoch 799/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1418 - acc: 1.0000
Epoch 800/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1417 - acc: 1.0000
E

## 複数層の場合

4層のニューラルネットワークは以下のように記述できます。

In [15]:
input_data = tf.keras.layers.Input(shape=(2,))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=input_data, outputs=output)



In [ ]:
# この記述方法では枝分かれを表現することもできます。以下は3層目で2つに枝分かれし、次の層で結合している例です。
input_data = tf.keras.layers.Input(shape=(2,))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
y1 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
y2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
z = tf.keras.layers.concatenate([y1, y2])
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(z)
model = tf.keras.Model(inputs=input_data, outputs=output)

## ラッパーとしてのKeras

ラッパーとしてのKerasもデフォルトでTensorFlowをバックエンドとして使用しているため、基本的な使い方は同じです。


ドキュメントが日本語でも公開されているため、tf.kerasを利用する上で参考にすることができます。例えば2つの記述方法については以下のページです。


compileメソッドで指定できる損失関数もまとまっています。


損失関数 - Keras Documentation


Sequentialモデルは以下のように書けます。ロジスティック回帰の例です。


以下のコードのほとんどは上で紹介したtf.kerasと実質的に同じですが、例えば活性化関数を全結合層とは別のクラスとして渡しています。

また、最適化手法の部分はtf.train.AdamOptimizerからkeras.optimizers.Adamに変わっています。tf.kerasではTensorFlow自体の最適化手法クラスを呼んでいるのに対し、

KerasではKeras独自の最適化手法クラスを使用するためです。ラッパーとしてのKerasのコードも見る機会が多いですから若干の違いに慣れておくと良いでしょう。

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

import numpy as np
# ANDゲートの学習データを用意
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

model = Sequential()
model.add(Dense(1, input_shape=(2,)))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

poch 801/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1331 - accuracy: 1.0000
Epoch 802/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1332 - accuracy: 1.0000
Epoch 803/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1328 - accuracy: 1.0000
Epoch 804/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.1325 - accuracy: 1.0000
Epoch 805/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1324 - accuracy: 1.0000
Epoch 806/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.1321 - accuracy: 1.0000
Epoch 807/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1322 - accuracy: 1.0000
Epoch 808/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1318 - accuracy: 1.0000
Epoch 809/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1316 - accuracy: 1.0000
Epoch 810/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.1314 - accuracy: 1.0000
E